### Automatic model selection

Test notebook for automatic model selection.

In [1]:
import datetime
import logging
import sys
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

#sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')
sys.path.append('/media/controlslab/DATA/Projects')

import automatic_model_selection
from automatic_model_selection import Configuration
from ann_encoding_rules import Layers
from CMAPSAuxFunctions import TrainValTensorBoard

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression
#from tunable_model import SequenceTunableModelRegression

#Data handlers
from ann_framework.data_handlers.data_handler_CMAPS import CMAPSDataHandler
from ann_framework.data_handlers.data_handler_MNIST import MNISTDataHandler
#from data_handler_MNIST import MNISTDataHandler
#from data_handler_CMAPS import CMAPSDataHandler

Using TensorFlow backend.


### Load cmaps data handler

In [2]:
def cmaps_dhandler():

    #Selected as per CNN paper
    features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 
    'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
    selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
    selected_features = list(features[i] for i in selected_indices-1)
    data_folder = '../CMAPSSData'

    window_size = 25
    window_stride = 1
    max_rul = 130

    dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)

    input_shape = (len(selected_features)*window_size, )

    return dHandler_cmaps, input_shape

### Function to save top models

In [3]:
def save_best_models(best_models_list, global_best_model_index, saveto, train_epochs=100):
    
    n_models = len(best_models_list)
    
    for ind_model, i in zip(best_models_list, range(n_models)):
        
        tModel = ind_model.tModel
        kmodel = tModel.model
        model_json = kmodel.to_json()
        
        #Save model's architecture
        string_append = str(i) if i != global_best_model_index else 'global'
        with open(saveto+"bestModel_"+string_append+".json", "w") as json_file:
            json_file.write(model_json)
            
    #Train the global best
    tModel = best_models_list[global_best_model_index].tModel
    print(tModel.data_handler.data_scaler)
        
    tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
    tModel.print_data()
    tModel.epochs = train_epochs

    tModel.train_model(verbose=1)
    tModel.evaluate_model(['rhs', 'rmse'], round=2)
    kmodel = tModel.model
            
    # serialize weights to HDF5
    kmodel.save_weights(saveto+"bestModel_global.h5")
    
    print("Saved models for dataset 1 to disk")
        

### Test on MNIST

In [4]:
"""Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
architecture_type = Layers.FullyConnected
problem_type = 2  #1 for regression, 2 for classification
output_shape = 10 #If regression applies, number of classes
input_shape = (784,)
pop_size = 5
tournament_size = 3
max_similar = 3
total_experiments = 5
count_experiments = 0
unroll = True

global_best_list = []
global_best = None
global_best_index = 0

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

t = datetime.datetime.now()

logging.basicConfig(filename='logs/nn_evolution_mnist_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                        format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

#cmaps datahandler
#dhandler_cmaps, input_shape = cmaps_dhandler()

#mnist datahandler
dHandler_mnist = MNISTDataHandler()

config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, tournament_size, max_similar, epochs=5, cross_val=0.2, size_scaler=0.4,
                       max_generations=10, binary_selection=True, mutation_ratio=0.4, similarity_threshold=0.2, more_layers_prob=0.8)

while count_experiments < total_experiments:
    print("Launching experiment {}".format(count_experiments+1))
    logging.info("Launching experiment {}".format(count_experiments+1))
    best = automatic_model_selection.run_experiment(config, dHandler_mnist, count_experiments + 1, 
                                                    unroll=unroll, verbose_data=0, tModel_scaler=min_max_scaler)

    global_best_list.append(best)

    if global_best == None:
        global_best = best
    else:
        if best.fitness < global_best.fitness:
            global_best = best
            global_best_index = count_experiments

    count_experiments =  count_experiments + 1

print("Global best list\n")
logging.info("Global best list\n")
automatic_model_selection.print_best(global_best_list)

print("Global best is\n")
print(global_best)
logging.info("Global best is\n")
logging.info(global_best)

save_best_models(global_best_list, global_best_index, 'best_models/mnist/', train_epochs=200)

Launching experiment 1

Generation 1
Fetching to keras
Evaluating population
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 816)               640560    
_________________________________________________________________
dropout_1 (Dropout)          (None, 816)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               418304    
_________________________________________________________________
dense_2 (Dense)              (None, 264)               135432    
_________________________________________________________________
dense_3 (Dense)              (None, 920)               243800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 920)               0         
_________________________________________________________________

/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


training with cv
Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 11s 237us/step - loss: 1.9130 - acc: 0.2655 - val_loss: 1.4125 - val_acc: 0.4753
Epoch 2/5
48000/48000 [==============================] - 11s 219us/step - loss: 1.1427 - acc: 0.6158 - val_loss: 0.8047 - val_acc: 0.7948
Epoch 3/5
48000/48000 [==============================] - 10s 218us/step - loss: 0.6914 - acc: 0.8297 - val_loss: 0.5017 - val_acc: 0.8821
Epoch 4/5
48000/48000 [==============================] - 11s 225us/step - loss: 0.4870 - acc: 0.8865 - val_loss: 0.3967 - val_acc: 0.9053
Epoch 5/5
12000/12000 [==============================] - 2s 176us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 104)               81640     
_________________________________________________________________
dropout_4 (Dropout)          (None, 104)      

training with cv
Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 5s 114us/step - loss: 0.7036 - acc: 0.7728 - val_loss: 0.3294 - val_acc: 0.9027
Epoch 2/5
48000/48000 [==============================] - 4s 92us/step - loss: 0.4033 - acc: 0.8768 - val_loss: 0.2821 - val_acc: 0.9166
Epoch 3/5
48000/48000 [==============================] - 4s 90us/step - loss: 0.3510 - acc: 0.8933 - val_loss: 0.2312 - val_acc: 0.9304
Epoch 4/5
48000/48000 [==============================] - 4s 89us/step - loss: 0.3108 - acc: 0.9063 - val_loss: 0.2289 - val_acc: 0.9335
Epoch 5/5
12000/12000 [==============================] - 1s 91us/step

Generating offsprings
Applying Mutation
Launch new generation?: True

Generation 2
Fetching to keras
Evaluating population
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 72)                56520  

12000/12000 [==============================] - 2s 158us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 776)               609160    
_________________________________________________________________
dense_32 (Dense)             (None, 48)                37296     
_________________________________________________________________
dropout_21 (Dropout)         (None, 48)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 776)               38024     
_________________________________________________________________
dropout_22 (Dropout)         (None, 776)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 10)                7770      
Total params: 692,250
Trainable params: 692,250
Non-trainable params: 0
_________

KeyboardInterrupt: 

### Test on CMAPSS

In [ ]:
"""Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
architecture_type = Layers.FullyConnected
problem_type = 1  #1 for regression, 2 for classification
output_shape = 1 #If regression applies, number of classes
input_shape = (784,)
pop_size = 5
tournament_size = 3
max_similar = 3
total_experiments = 5
count_experiments = 0
unroll = True

global_best_list = []
global_best = None
global_best_index = 0

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

t = datetime.datetime.now()

logging.basicConfig(filename='logs/nn_evolution_cmaps_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                        format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

#cmaps datahandler
dhandler_cmaps, input_shape = cmaps_dhandler()
print(input_shape)

#mnist datahandler
#dHandler_mnist = MNISTDataHandler()

config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, tournament_size, max_similar, epochs=5, cross_val=0.2, size_scaler=0.4,
                       max_generations=10, binary_selection=True, mutation_ratio=0.4, similarity_threshold=0.2, more_layers_prob=0.8)

while count_experiments < total_experiments:
    print("Launching experiment {}".format(count_experiments+1))
    logging.info("Launching experiment {}".format(count_experiments+1))
    best = automatic_model_selection.run_experiment(config, dhandler_cmaps, count_experiments + 1, 
                                                    unroll=unroll, verbose_data=0, tModel_scaler=min_max_scaler)

    global_best_list.append(best)

    if global_best == None:
        global_best = best
    else:
        if best.fitness < global_best.fitness:
            global_best = best
            global_best_index = count_experiments

    count_experiments =  count_experiments + 1

print("Global best list\n")
logging.info("Global best list\n")
automatic_model_selection.print_best(global_best_list)

print("Global best is\n")
print(global_best)
logging.info("Global best is\n")
logging.info(global_best)

save_best_models(global_best_list, global_best_index, 'best_models/cmapss/', train_epochs=200)

In [ ]:
#save_best_models(global_best_list, global_best_index, train_epochs=200)